In [1]:
#1. build our own dataset

In [38]:
from datasets import Dataset, DatasetDict
from transformers import DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np

# map POS to POS_id
cnt = 0
POS_id = {}
POS_ls = ['NN', 'IN', 'NNP', 'DT', 'NNS', 'JJ', 'COMMA', 'CD', '.', 'VBD', 'RB','VB', 'CC', 'VBN', 'VBZ', 
          'VBG', 'TO', 'PRP', 'VBP', 'POS', 'PRP$','MD', '$', '``', "''", 'WDT', ':', 'JJR', 'RP', 'RBR', 
          'WP', 'NNPS','JJS', ')', '(', 'EX', 'RBS', 'WRB', '-', 'UH', 'WP$', 'PDT', '/', '#', 'LS', 'SYM', 'FW', 'AUX']
for pos in POS_ls:
    POS_id[pos] = cnt
    cnt += 1

# map BIO to BIO_id
cnt = 0
BIO_id = {}
BIO_ls = ['O', 'B-NP', 'I-NP', 'B-PP', 'B-ADVP', 'B-ADJP', 'B-SBAR', 'B-CONJP',
       'I-ADJP', 'I-PP', 'I-ADVP', 'I-CONJP', 'B-INTJ', 'I-SBAR', 'B-LST',
       'B-VP', 'B-PRT', 'I-INTJ', 'I-VP']
for bio in BIO_ls:
    BIO_id[bio] = cnt
    cnt += 1

In [266]:


# map label to ARG_id
Label_id = {"ARG0":0,"ARG1":1,"ARG2":2,"PRED":3,"SUPPORT":4}

def mapLabel(label):
    return Label_id[label] if label in Label_id else 5

def mapPOS(label):
    return POS_id[label] if label in POS_id else len(POS_id)+1

def mapBIO(label):
    return BIO_id[label] if label in BIO_id else len(BIO_id)+1

# build datasets
def condense_df(file):
    df = pd.DataFrame()
    with open(file, 'r') as file:
        ls = [i.split('\t') for i in file.read().split('\n')]
        df = pd.DataFrame(ls)

    df['id'] = df.index
    df[0].replace('', np.nan, inplace=True)
    df.dropna(axis=0, subset = [0], inplace = True)
    df['BIO'] = df[2].map(mapBIO)
    df['POS'] = df[1].map(mapPOS)
    df['label'] = df[5].map(mapLabel)
    df['id'] = df[4].map(int)
    df.drop(columns = [1, 2, 3, 4, 5, 6], inplace = True)
    condense = df.groupby('id').apply(lambda x: [list(x[0]),list(x['POS']), list(x['BIO']), list(x['label'])]).apply(pd.Series)
    condense.columns =['tokens','BIO', 'POS','label']
    return condense


train = Dataset.from_pandas(condense_df("Partitive-Files/%_nombank.clean.train"))
eval_ = Dataset.from_pandas(condense_df("Partitive-Files/%_nombank.clean.dev"))
test = Dataset.from_pandas(condense_df("Partitive-Files/%_nombank.clean.test"))


datasets = DatasetDict({"train": train, "validation":eval_, "test":test})
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'BIO', 'POS', 'label', 'id'],
        num_rows: 2174
    })
    validation: Dataset({
        features: ['tokens', 'BIO', 'POS', 'label', 'id'],
        num_rows: 83
    })
    test: Dataset({
        features: ['tokens', 'BIO', 'POS', 'label', 'id'],
        num_rows: 150
    })
})

In [40]:
# 2. tokenize

In [311]:

from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def align_labels_with_tokens(labels, POSs, BIOs, word_ids):
    new_labels = []
    POS_labels = []
    BIO_labels = []
    current_word = None
    for word_id in word_ids:
        if not word_id:
            new_labels.append(-100)
            POS_labels.append(0)
            BIO_labels.append(0)
        else:
            if word_id != current_word:# Start of a new word!
                current_word = word_id       
            new_labels.append(labels[word_id])
            POS_labels.append(POSs[word_id])
            BIO_labels.append(BIOs[word_id])

    return new_labels, POS_labels, BIO_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    all_BIO = examples["BIO"]
    all_POS = examples["POS"]
    all_labels = examples["label"]
    
    new_labels = []
    POS_labels = []
    BIO_labels = []
    for i in range(len(all_labels)):
        word_ids = tokenized_inputs.word_ids(i)
        new_label, POS_label, BIO_label = align_labels_with_tokens(all_labels[i], all_POS[i], all_BIO[i], word_ids)
        new_labels.append(new_label)
        POS_labels.append(POS_label)
        BIO_labels.append(BIO_label)

    tokenized_inputs["BIOL"] = BIO_labels
    tokenized_inputs["POSL"] = POS_labels
    tokenized_inputs["labels"] = new_labels
    #print("new_labels::"+str(new_labels))
    #print("POS_labels::"+str(POS_labels))

    return tokenized_inputs

tokenized_datasets = datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns = datasets["train"].column_names,
)
"""
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if not word_id:
            new_labels.append(-100)
        else:
            if word_id != current_word:# Start of a new word!
                current_word = word_id       
            label = labels[word_id]
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    all_labels = examples["label"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

tokenized_datasets = datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns = datasets["train"].column_names,
)
"""

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

'\nfrom transformers import AutoTokenizer\n\nmodel_checkpoint = "bert-base-uncased"\ntokenizer = AutoTokenizer.from_pretrained(model_checkpoint)\n\ndef align_labels_with_tokens(labels, word_ids):\n    new_labels = []\n    current_word = None\n    for word_id in word_ids:\n        if not word_id:\n            new_labels.append(-100)\n        else:\n            if word_id != current_word:# Start of a new word!\n                current_word = word_id       \n            label = labels[word_id]\n            new_labels.append(label)\n\n    return new_labels\n\ndef tokenize_and_align_labels(examples):\n    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)\n    all_labels = examples["label"]\n    new_labels = []\n    for i, labels in enumerate(all_labels):\n        word_ids = tokenized_inputs.word_ids(i)\n        new_labels.append(align_labels_with_tokens(labels, word_ids))\n\n    tokenized_inputs["labels"] = new_labels\n    return tokenized_inputs\n\

In [ ]:
# 3. train while evaluate

In [312]:

import evaluate
import numpy as np

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = evaluate.load("seqeval")

small_train = tokenized_datasets["train"].shuffle(seed=42).select(range(64))
small_eval = tokenized_datasets["validation"].shuffle(seed=42).select(range(16))
small_test = tokenized_datasets["test"].shuffle(seed=42).select(range(8))


BatchSize = 1
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
    small_train, shuffle=True, batch_size=BatchSize, collate_fn=data_collator#tokenized_datasets["train"]
)
eval_dataloader = DataLoader(
    small_eval, batch_size=BatchSize, collate_fn=data_collator#tokenized_datasets["validation"]
)
test_dataloader = DataLoader(
    small_test, batch_size=BatchSize, collate_fn=data_collator#tokenized_datasets["test"]
)

In [ ]:
#len(small_train['POS'])

In [ ]:
#len(small_train['POS'][2])

In [337]:
label_names = ["ARG0", "ARG1", "ARG2", "PRED", "SUPPORT", "None"]
POS_len = len(POS_ls)
BIO_len = len(BIO_ls)
feature_dim = 768

In [338]:
from transformers import AutoConfig, AutoModel
from transformers.modeling_outputs import TokenClassifierOutput

import torch.nn as nn
class CustomModel(nn.Module):
    def __init__(self,checkpoint,num_labels): 
        super(CustomModel,self).__init__() 
        self.num_labels = num_labels 

        #Load Model with given checkpoint and extract its body
        self.model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
        self.dropout = nn.Dropout(0.1) 
        self.classifier = nn.Linear(feature_dim,num_labels) # load and initialize weights

    def forward(self, input_ids, token_type_ids, attention_mask, labels, POS, BIO):
        #TODO: add 2 more parameters as: forward(self, input_ids, token_type_ids, attention_mask, POS, BIO, labels)
        #Extract outputs from the body
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
#         print("input_ids:::"+str(input_ids.shape))
#         print("labels:::"+str(labels.shape))
        #Add custom layers
        sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state
        
#         POS_f = torch.eye(POS_len)[POS]
#         BIO_f = torch.eye(POS_len)[BIO]
        
#         sequence_output = torch.cat((sequence_output,POS_f, BIO_f),2)
        #sequence_output = outputs
#         print(sequence_output.size())
        """
        TODO: concat
        After using print(sequence_output.size()) here, we have:
        torch.Size([8, 44, 768])
        torch.Size([8, 50, 768]) etc.
        第一维度是全局batch_size我设置成8
        第二维是一句话里面token的个数，是经过分割OOV过后的token数量
        第三维是Bert的输出
        
        假设POS 48维，BIO 10维，那么总的第三维在concat后的总维数是 768+48+10，concat不会改变第一维和第二维的长度
        
        logits = self.classifier(sequence_output[:,:,:].view(-1, 768+48+10)) # calculate losses
        """
        
        logits = self.classifier(sequence_output[:,:,:].view(-1, feature_dim)) # calculate losses
        self.logits = logits
        #print("logits::"+str(logits.shape))

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

In [339]:
import torch
from tqdm import tqdm
from transformers import AdamW,get_scheduler
from datasets import load_metric
metric = evaluate.load("seqeval")

num_epochs = 10
model_cc = CustomModel(checkpoint=model_checkpoint,num_labels=len(label_names))#.cuda()
optimizer = AdamW(model_cc.parameters(), lr=2e-5)
num_training_steps = num_epochs * len(train_dataloader)
progress_bar_train = tqdm(range(num_training_steps),miniters=2)
progress_bar_eval = tqdm(range(num_epochs * len(eval_dataloader)),miniters=2)
f1_best = 0
resume_flag = False
best_net = None

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

for epoch in range(num_epochs):
    if resume_flag:
        model_cc.load_state_dict(torch.load("sstcls_best.dat"))
    model_cc.train()
    for batch in train_dataloader:
        batch = {k: v for k, v in batch.items()}#.cuda()
        outputs = model_cc(batch['input_ids'], token_type_ids=batch['token_type_ids'], attention_mask=batch['attention_mask'],labels=batch['labels'], POS=batch['POSL'], BIO=batch['BIOL'])
#         assert()
        #TODO: add 2 more parameters as followed:
        #model_cc(batch['input_ids'], token_type_ids=batch['token_type_ids'], attention_mask=batch['attention_mask'],POS = batch['POS'], BIO = batch['BIO'], labels=batch['labels'])
        
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)
    model_cc.eval()
    f1_now = []
    for batch in eval_dataloader:
        batch = {k: v for k, v in batch.items()}#.cuda()
        with torch.no_grad():
            outputs = model_cc(batch['input_ids'], token_type_ids=batch['token_type_ids'], attention_mask=batch['attention_mask'],labels=batch['labels'], POS=batch['POSL'], BIO=batch['BIOL'])
            #TODO: add 2 more parameters as followed:
            #model_cc(batch['input_ids'], token_type_ids=batch['token_type_ids'], attention_mask=batch['attention_mask'],POS = batch['POS'], BIO = batch['BIO'], labels=batch['labels'])
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        true_labels = [[label_names[l] for l in label if l != -100] for label in batch["labels"]]
        
        reshaped_predictions = torch.reshape(predictions, (BatchSize,-1))
        true_predictions = [
            [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(reshaped_predictions, batch["labels"])
        ]
        
        all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
        metric.add_batch(predictions=true_predictions, references=true_labels)
        progress_bar_eval.update(1)
        f1_now.append(all_metrics["overall_f1"])
        
    if np.mean(f1_now) > f1_best or not best_net:
        torch.save(model_cc.state_dict(), 'sstcls_best.dat')
        f1_best = np.mean(f1_now)
        print("the best f1 is now: "+ str(np.mean(f1_now)))
        best_net = model_cc
        

    print(metric.compute())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

  0%|                                                  | 0/640 [00:35<?, ?it/s]


  0%|                                                  | 0/160 [00:35<?,

the best f1 is now: 0.029824561403508774
{'RED': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'RG1': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'UPPORT': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'one': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 3}, 'overall_precision': 0.0, 'overall_recall': 0.0, 'overall_f1': 0.0, 'overall_accuracy': 0.9142857142857143}



 20%|████████                                | 128/640 [01:33<04:55,  1.74it/s]

 11%|████▎                                    | 17/160 [01:35<30:48, 12.92s/it]

 11%|████▌                                    | 18/160 [01:36<22:08,  9.36s/it]

 12%|████▊                                    | 19/160 [01:37<16:07,  6.86s/it]

 12%|█████▏                                   | 20/160 [01:38<11:57,  5.12s/it]

 13%|█████▍                                   | 21/160 [01:40<09:02,  3.90s/it]

 14%|█████▋                                   | 22/160 [01:41<07:00,  3.05s/it]

 14%|█████▉                                   | 23/160 [01:42<05:35,  2.45s/it]

 15%|██████▏                                  | 24/160 [01:43<04:36,  2.03s/it]

 16%|██████▍                                  | 25/160 [01:44<03:54,  1.74s/it]

 16%|██████▋                                  | 26/160 [01:45<03:25,  1.53s/it]

 17%|██████▉                                  | 27/160 [01:46<03:04,  1.39s/it]

 18%|███████▏              

the best f1 is now: 0.4471864385446294
{'RED': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1}, 'RG1': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'UPPORT': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'one': {'precision': 0.5, 'recall': 0.3333333333333333, 'f1': 0.4, 'number': 3}, 'overall_precision': 0.6666666666666666, 'overall_recall': 0.3333333333333333, 'overall_f1': 0.4444444444444444, 'overall_accuracy': 0.9428571428571428}



 30%|████████████                            | 192/640 [02:30<04:18,  1.73it/s]

 21%|████████▍                                | 33/160 [02:32<27:24, 12.95s/it]

 21%|████████▋                                | 34/160 [02:33<19:42,  9.38s/it]

 22%|████████▉                                | 35/160 [02:34<14:20,  6.88s/it]

 22%|█████████▏                               | 36/160 [02:35<10:36,  5.14s/it]

 23%|█████████▍                               | 37/160 [02:36<08:00,  3.91s/it]

 24%|█████████▋                               | 38/160 [02:37<06:12,  3.05s/it]

 24%|█████████▉                               | 39/160 [02:38<04:57,  2.46s/it]

 25%|██████████▎                              | 40/160 [02:39<04:04,  2.04s/it]

 26%|██████████▌                              | 41/160 [02:40<03:27,  1.74s/it]

 26%|██████████▊                              | 42/160 [02:41<03:01,  1.53s/it]

 27%|███████████                              | 43/160 [02:42<02:42,  1.39s/it]

 28%|███████████▎          

...

{'RED': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1}, 'RG1': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'UPPORT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1}, 'one': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1': 0.6666666666666666, 'number': 3}, 'overall_precision': 0.8, 'overall_recall': 0.6666666666666666, 'overall_f1': 0.7272727272727272, 'overall_accuracy': 0.9714285714285714}



 90%|████████████████████████████████████    | 576/640 [08:03<00:36,  1.74it/s]

 81%|████████████████████████████████▎       | 129/160 [08:05<06:26, 12.47s/it]

 81%|████████████████████████████████▌       | 130/160 [08:06<04:31,  9.05s/it]

 82%|████████████████████████████████▊       | 131/160 [08:07<03:12,  6.65s/it]

 82%|█████████████████████████████████       | 132/160 [08:08<02:19,  4.97s/it]

 83%|█████████████████████████████████▎      | 133/160 [08:09<01:42,  3.80s/it]

 84%|█████████████████████████████████▌      | 134/160 [08:10<01:17,  2.97s/it]

 84%|█████████████████████████████████▊      | 135/160 [08:11<00:59,  2.40s/it]

 85%|██████████████████████████████████      | 136/160 [08:12<00:47,  1.99s/it]

 86%|██████████████████████████████████▎     | 137/160 [08:13<00:39,  1.71s/it]

 86%|██████████████████████████████████▌     | 138/160 [08:14<00:33,  1.52s/it]

 87%|██████████████████████████████████▊     | 139/160 [08:15<00:28,  1.38s/it]

 88%|██████████████████████

{'RED': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1}, 'RG1': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'UPPORT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1}, 'one': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1': 0.6666666666666666, 'number': 3}, 'overall_precision': 0.8, 'overall_recall': 0.6666666666666666, 'overall_f1': 0.7272727272727272, 'overall_accuracy': 0.9714285714285714}



100%|████████████████████████████████████████| 640/640 [08:58<00:00,  1.73it/s]

 91%|████████████████████████████████████▎   | 145/160 [09:00<03:07, 12.53s/it]

 91%|████████████████████████████████████▌   | 146/160 [09:01<02:07,  9.09s/it]

 92%|████████████████████████████████████▊   | 147/160 [09:02<01:26,  6.68s/it]

 92%|█████████████████████████████████████   | 148/160 [09:03<00:59,  4.99s/it]

 93%|█████████████████████████████████████▎  | 149/160 [09:04<00:41,  3.81s/it]

 94%|█████████████████████████████████████▌  | 150/160 [09:05<00:29,  2.98s/it]

 94%|█████████████████████████████████████▊  | 151/160 [09:06<00:21,  2.41s/it]

 95%|██████████████████████████████████████  | 152/160 [09:07<00:16,  2.00s/it]

 96%|██████████████████████████████████████▎ | 153/160 [09:08<00:12,  1.72s/it]

 96%|██████████████████████████████████████▌ | 154/160 [09:09<00:09,  1.52s/it]

 97%|██████████████████████████████████████▊ | 155/160 [09:10<00:06,  1.38s/it]

 98%|██████████████████████

{'RED': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1}, 'RG1': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'UPPORT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1}, 'one': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1': 0.6666666666666666, 'number': 3}, 'overall_precision': 0.8, 'overall_recall': 0.6666666666666666, 'overall_f1': 0.7272727272727272, 'overall_accuracy': 0.9714285714285714}


In [340]:
def predict(net, dataloader, gpu):
    net.eval()

    f1_now = []
    precision_now = []
    recall_now = []
    accuracy_now = []
    for batch in eval_dataloader:
        batch = {k: v for k, v in batch.items()}#.cuda()
        with torch.no_grad():
            outputs = model_cc(batch['input_ids'], token_type_ids=batch['token_type_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'], POS=batch['POSL'], BIO=batch['BIOL'])
            #TODO: add 2 more parameters as followed:
            #model_cc(batch['input_ids'], token_type_ids=batch['token_type_ids'], attention_mask=batch['attention_mask'],POS = batch['POS'], BIO = batch['BIO'], labels=batch['labels'])
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        true_labels = [[label_names[l] for l in label if l != -100] for label in batch["labels"]]
        
        reshaped_predictions = torch.reshape(predictions, (BatchSize,-1))
        true_predictions = [
            [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(reshaped_predictions, batch["labels"])
        ]
        
        all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
        metric.add_batch(predictions=true_predictions, references=true_labels)
        progress_bar_eval.update(1)
        f1_now.append(all_metrics["overall_f1"])
        precision_now.append(all_metrics["overall_precision"])
        recall_now.append(all_metrics["overall_recall"])
        accuracy_now.append(all_metrics["overall_accuracy"])


    return np.mean(f1_now), np.mean(precision_now), np.mean(recall_now), np.mean(accuracy_now),



In [341]:
preds = predict(best_net, test_dataloader, 0)
print("test_f1_score:::"+str(preds[0]))
print("test_precision_score:::"+str(preds[1]))
print("test_recall_score:::"+str(preds[2]))
print("test_accuracy_score:::"+str(preds[3]))



161it [09:18,  1.41s/it]                                                       

162it [09:19,  1.30s/it]

163it [09:20,  1.23s/it]

164it [09:21,  1.18s/it]

165it [09:22,  1.14s/it]

166it [09:23,  1.11s/it]

167it [09:24,  1.10s/it]

168it [09:25,  1.08s/it]

169it [09:26,  1.07s/it]

170it [09:27,  1.07s/it]

171it [09:28,  1.06s/it]

172it [09:29,  1.06s/it]

173it [09:30,  1.06s/it]

174it [09:31,  1.06s/it]

175it [09:32,  1.06s/it]

176it [09:34,  1.06s/it]

test_f1_score:::0.7078461652481628
test_precision_score:::0.6690595698348256
test_recall_score:::0.776018772893773
test_accuracy_score:::0.9446854659849268
